## 1. Setup e Importações

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Bibliotecas carregadas")

## 2. Carregamento dos Dados

In [ ]:
# Carrega dados brutos
raw_df = pd.read_excel('data/raw/Mega-Sena.xlsx')
raw_df['data'] = pd.to_datetime(raw_df['data'], format='mixed', dayfirst=True)

# Carrega features observadas
features_df = pd.read_parquet('data/processed/draws_features.parquet')

# Carrega simulação Monte Carlo
simulation_df = pd.read_parquet('data/processed/monte_carlo_simulation.parquet')

# Carrega estatísticas do baseline
baseline_stats = pd.read_parquet('data/processed/baseline_statistics.parquet')

# Carrega resultados da validação
validation_df = pd.read_parquet('data/processed/validation_results.parquet')

# Carrega resumo da validação
with open('data/processed/validation_summary.json', 'r') as f:
    validation_summary = json.load(f)

print(f"✓ Dados brutos: {len(raw_df)} concursos")
print(f"✓ Features observadas: {len(features_df)} concursos, {len(features_df.columns)-2} features")
print(f"✓ Simulação Monte Carlo: {len(simulation_df)} sorteios simulados")
print(f"✓ Validação: {len(validation_df)} features testadas")

## 3. Estatísticas Descritivas

In [ ]:
print("="*70)
print("INFORMAÇÕES GERAIS")
print("="*70)
print(f"Período analisado: {raw_df['data'].min().date()} a {raw_df['data'].max().date()}")
print(f"Total de concursos: {len(raw_df):,}")
print(f"Total de sorteios simulados: {len(simulation_df):,}")
print(f"Simulações Monte Carlo: {len(simulation_df) // len(raw_df):,}")
print(f"\nFeatures espaciais calculadas: {len(validation_df)}")
print("\nCategorias de features:")
print("  - Posicionais: centroid_row, centroid_col, dispersion")
print("  - Quadrantes: q1, q2, q3, q4")
print("  - Bordas: border_count, corner_count")
print("  - Distribuições: row_min, row_max, col_min, col_max, row_std, col_std")
print("  - Topológicas: adjacencies_4, adjacencies_8, connectivity_4, connectivity_8")
print("  - Geométricas: inertia_row, inertia_col, eccentricity, compactness")
print("  - Simetria: symmetry_horizontal, symmetry_vertical")
print("  - Anéis: ring_center, ring_middle, ring_outer")

## 4. Resumo da Validação Estatística

In [ ]:
print("="*70)
print("RESULTADOS DA VALIDAÇÃO ESTATÍSTICA")
print("="*70)
print(f"\nMétodo de correção: {validation_summary['correction_method']}")
print(f"Total de features testadas: {validation_summary['n_features_tested']}")
print(f"Features significativas (p < 0.05): {validation_summary['n_significant']}")
print(f"Features com efeito grande (|z| ≥ 0.5): {validation_summary['n_large_effect']}")
print(f"Features com efeito médio (0.2 ≤ |z| < 0.5): {validation_summary['n_medium_effect']}")

if validation_summary['significant_features']:
    print("\nFeatures significativas:")
    for feat in validation_summary['significant_features']:
        print(f"  - {feat}")
else:
    print("\n⚠️  Nenhuma feature significativa encontrada.")
    print("\n✅ CONCLUSÃO: Os sorteios da Mega-Sena são compatíveis com aleatoriedade pura.")
    print("   Não há evidências de padrões espaciais nos dados observados.")

## 5. Top 10 Features por Tamanho de Efeito

In [ ]:
# Ordena por effect size
top_features = validation_df.sort_values('effect_size', ascending=False).head(10)

# Exibe tabela formatada
display_cols = ['feature', 'effect_size', 'p_value', 'p_value_adjusted', 'significant', 'effect_interpretation']
print("\nTop 10 Features por Tamanho de Efeito:")
print("="*100)
print(top_features[display_cols].to_string(index=False))
print("="*100)

## 6. Comparação Observado vs Baseline (Tabela Detalhada)

In [ ]:
# Prepara comparação
comparison = validation_df[[
    'feature', 
    'observed_mean', 
    'simulated_mean', 
    'difference',
    'difference_pct',
    'ci_lower', 
    'ci_upper',
    'outside_ci_95'
]].copy()

comparison = comparison.sort_values('feature')

print("\nComparação Observado vs Baseline (Monte Carlo):")
print("="*120)
print(comparison.to_string(index=False))
print("="*120)
print(f"\nFeatures fora do IC 95%: {comparison['outside_ci_95'].sum()}")

## 7. Visualizações

### 7.1 Heatmap de Densidade do Volante

In [ ]:
from IPython.display import Image, display

display(Image('reports/heatmap_density.png'))

### 7.2 Comparação de Dispersão

In [ ]:
display(Image('reports/dispersion_comparison.png'))

### 7.3 Distribuição de Centroides

In [ ]:
display(Image('reports/centroid_scatter.png'))

### 7.4 Comparação de Todas as Features

In [ ]:
display(Image('reports/feature_comparison.png'))

### 7.5 Distribuição de Tamanhos de Efeito

In [ ]:
display(Image('reports/effect_size_distribution.png'))

## 8. Análise Interativa - Explore Features Específicas

In [ ]:
# Selecione uma feature para análise detalhada
feature_name = 'dispersion'  # Altere aqui para explorar outras features

# Dados observados vs simulados
obs_values = features_df[feature_name]
sim_values = simulation_df[feature_name]

# Estatísticas
print(f"Análise detalhada: {feature_name}")
print("="*70)
print(f"\nObservado:")
print(f"  Média: {obs_values.mean():.4f}")
print(f"  Desvio padrão: {obs_values.std():.4f}")
print(f"  Min: {obs_values.min():.4f}, Max: {obs_values.max():.4f}")

print(f"\nSimulado (Baseline):")
print(f"  Média: {sim_values.mean():.4f}")
print(f"  Desvio padrão: {sim_values.std():.4f}")
print(f"  Min: {sim_values.min():.4f}, Max: {sim_values.max():.4f}")

# Validação
feat_validation = validation_df[validation_df['feature'] == feature_name].iloc[0]
print(f"\nValidação Estatística:")
print(f"  P-valor: {feat_validation['p_value']:.4f}")
print(f"  P-valor ajustado (FDR): {feat_validation['p_value_adjusted']:.4f}")
print(f"  Effect size: {feat_validation['effect_size']:.4f}")
print(f"  Significativo: {feat_validation['significant']}")
print(f"  Interpretação: {feat_validation['effect_interpretation']}")

# Gráfico comparativo
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(sim_values, bins=50, alpha=0.7, label='Simulado (Baseline)', density=True, color='gray')
ax.hist(obs_values, bins=30, alpha=0.7, label='Observado', density=True, color='blue')

# Linhas de média
ax.axvline(sim_values.mean(), color='gray', linestyle='--', linewidth=2, label=f'Média Simulado: {sim_values.mean():.3f}')
ax.axvline(obs_values.mean(), color='blue', linestyle='--', linewidth=2, label=f'Média Observado: {obs_values.mean():.3f}')

# IC 95%
ci_lower = feat_validation['ci_lower']
ci_upper = feat_validation['ci_upper']
ax.axvspan(ci_lower, ci_upper, alpha=0.2, color='red', label=f'IC 95% [{ci_lower:.3f}, {ci_upper:.3f}]')

ax.set_xlabel(feature_name, fontsize=12)
ax.set_ylabel('Densidade', fontsize=12)
ax.set_title(f'Distribuição: {feature_name}', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Estatísticas do Baseline (Monte Carlo)

In [ ]:
print("Estatísticas do Baseline (5000 simulações):")
print("="*100)
print(baseline_stats.to_string())
print("="*100)

## 10. Conclusões e Interpretação

### Principais Achados:

1. **Validação Estatística**: Nenhuma das 27 features espaciais testadas apresentou diferença estatisticamente significativa do baseline aleatório (p < 0.05 após correção FDR).

2. **Tamanhos de Efeito**: Todos os effect sizes foram desprezíveis (< 0.05), indicando que mesmo as diferenças observadas são triviais.

3. **Distribuições Espaciais**: As distribuições de centroides, dispersão, quadrantes e outras métricas espaciais são indistinguíveis de sorteios puramente aleatórios.

### Interpretação:

**✅ Os sorteios da Mega-Sena são compatíveis com aleatoriedade pura.**

Não há evidências de:
- Viés espacial no volante
- Preferência por quadrantes específicos
- Padrões de adjacência ou agrupamento
- Simetrias ou assimetrias sistemáticas
- Qualquer outro padrão espacial detectável

### Metodologia Robusta:

- **5000 simulações Monte Carlo** com 2954 sorteios cada (~14.7 milhões de sorteios)
- **Correção FDR** (False Discovery Rate) para múltiplas comparações
- **27 features espaciais** cobrindo múltiplas dimensões da distribuição espacial
- **Testes estatísticos robustos** (Mann-Whitney U, Kolmogorov-Smirnov)

### Implicação Prática:

Não há base estatística para estratégias de jogo baseadas em padrões espaciais do volante. Os resultados confirmam a integridade e aleatoriedade dos sorteios.

---

**Análise completa. Para re-executar o pipeline completo:**

```bash
python -m src.pipeline run-all --n-simulations 5000
```